# Analyze Gaia image stack & add a logo

In [ ]:
from pathlib import Path
from pkg_resources import resource_filename
import matplotlib.pyplot as plt

import astro3d
from astro3d import image_stack

## Read the image stack

We again need to set the printer-specific settings and the directory of the slides:

In [ ]:
stack = image_stack.IStack(
    #astro3d.get_output('slices_gaia_2022_linear'),
    astro3d.get_output('slices_gaia_2022_log'))

## Print out statistics


In [ ]:
f, ax = stack.show_colors(titles=['Black', 'Red', 'PureWhite'], fontsize='small')

Now we assign the transparent color index

In [ ]:
stack.empty_indices = [0]

In [ ]:
stack.show_info()

## Plots

Show the distribution of materials

In [ ]:
f, ax = stack.show_counts()

Show fully transparent pixels and the density distribution.

In [ ]:
f, ax = stack.show_transparency_estimate()

All sides

In [ ]:
f, ax = stack.three_views()

## Add logo and more 

First we add the logo from an image file

In [ ]:
logo = resource_filename('astro3d', str(Path('..') / 'figures' / 'Logo_Origins+qr.png'))
# stack.add_logo(logo, [0.3, 0.3, 0.3], width=2.0, depth=0.09, col=image_stack.BaseCyan)

This one needs to be modified: the QR will get a white background, so we separate both parts and include them separately

In [ ]:
from PIL import Image
im = np.array(Image.open(logo)).transpose(1, 0, 2)[:, ::-1, :].max(-1)

x0 = 1125
x1 = 1979

ori = im[:x0, :]
qr = im[x0:x1, 20:-20]

f, ax = plt.subplots(1, 2)
ax[0].pcolormesh(np.arange(ori.shape[0]), np.arange(ori.shape[1]), ori.T, cmap='gray_r')
ax[1].pcolormesh(np.arange(qr.shape[0]), np.arange(qr.shape[1]), qr.T, cmap='gray_r')
ax[0].set_aspect('equal')
ax[1].set_aspect('equal')

In [ ]:
height = 1.2
width = ori.shape[0] / ori.shape[1] * height

stack.add_logo(ori, [0.3, 0.3, 0.3],         None, 0.09, height=height, threshold=0.3, pal=[stack.colors[0], image_stack.BaseCyan])
stack.add_logo(qr,  [0.4 + width, 0.3, 0.3], None, 0.09, height=height, threshold=0.3, pal=[image_stack.BaseWhite, image_stack.BaseCyan])

In [ ]:
stack.show_view('xz');

We mark the center with a 1mm-diameter sphere

In [ ]:
stack.add_sphere(np.array(stack.dimension) / 2, radius=0.05)

We add a scale bar. The full box is 740 x 740 x 540 pc. A 200 pc line is therefore $5 / 740 \times 200 \simeq 1.352$ cm.

In [ ]:
L = 5 / 740 * 200
pos = np.array([4.0, 0.3, 0.3])
stack.add_scale(L, pos, radius=12 * stack.dx)

In [ ]:
stack.add_text('200 parsec', pos + [0.05 * L, 0.0, 0.1 * L], 1.2, 0.09, col=[255, 0, 0])

We redo the rendering

In [ ]:
stack.show_colors();
stack.empty_indices = [1]

In [ ]:
stack.replace_color(0, [128, 128, 128])

In [ ]:
stack.show_colors();

In [ ]:
stack.replace_color(0, [255, 255, 255])

In [ ]:
stack.show_colors();
stack.empty_indices = [1]

In [ ]:
stack.show_view('xz');

In [ ]:
stack.three_views();

Save it under a new folder

In [ ]:
out_dir = stack.directory.parent / (stack.directory.name + '_logo')
!mkdir -p {out_dir}
stack.save_images(out_dir)
!open {out_dir}